In [1]:
'''
Simple version for the H2O_tutorial_eeg_eyestate.ipynb in H2o repo
'''

import h2o
import pandas as pd
import seaborn as sns
import imp
import math
import numpy as np

In [2]:
h2o.init()

/Users/lzq857/anaconda/lib/python2.7/site-packages/h2o/backend/connection.py:312: UserWarning: Proxy is defined in the environment: http_proxy. This may interfere with your H2O Connection.
  "This may interfere with your H2O Connection." % name)
/Users/lzq857/anaconda/lib/python2.7/site-packages/h2o/backend/connection.py:312: UserWarning: Proxy is defined in the environment: HTTP_PROXY. This may interfere with your H2O Connection.
  "This may interfere with your H2O Connection." % name)


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,5 mins 32 secs
H2O cluster version:,3.10.4.3
H2O cluster version age:,2 months and 6 days
H2O cluster name:,H2O_from_python_LZQ857_addx5e
H2O cluster total nodes:,1
H2O cluster free memory:,3.245 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [4]:
'''
Use GBC for EEG Eye State Classification: '1' indicates the eye-closed and '0' the eye-open state
'''

#csv_url = "http://www.stat.berkeley.edu/~ledell/data/eeg_eyestate_splits.csv"
#df = h2o.import_file(csv_url)

df = h2o.import_file(path='./eeg_eyestate_splits.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
df.shape

(14980, 16)

In [6]:
df.describe()

Rows:14980
Cols:16




,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,eyeDetection,split
type,real,real,real,real,real,real,real,real,real,real,real,real,real,real,int,enum
mins,1030.77,2830.77,1040.0,2453.33,2089.74,2768.21,2086.15,4567.18,1357.95,1816.41,3273.33,2257.95,86.6667,1366.15,0.0,
mean,4321.91777704,4009.76769359,4264.02243258,4164.94632644,4341.74107543,4644.02237917,4110.40015955,4616.05690387,4218.82661015,4231.3161996,4202.45689987,4279.23277437,4615.20533556,4416.43583244,0.448798397864,
maxs,309231.0,7804.62,6880.51,642564.0,6474.36,362564.0,567179.0,7264.1,265641.0,6674.36,6823.08,7002.56,152308.0,715897.0,1.0,
sigma,2492.07217427,45.9416724848,44.4280517574,5216.4046323,34.7388208185,2924.78953733,4600.92654253,29.2926032018,2136.40852289,38.0509026212,37.785981374,41.5443115167,1208.36995826,5891.28504252,0.497388088873,
zeros,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8257,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,4329.23,4009.23,4289.23,4148.21,4350.26,4586.15,4096.92,4641.03,4222.05,4238.46,4211.28,4280.51,4635.9,4393.85,0.0,valid
1,4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.1,0.0,test
2,4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,0.0,train


In [7]:
# The dataset has a "split" column, let's drop it and do our own split.
df = df.drop("split")
df.head()

AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,eyeDetection
4329.23,4009.23,4289.23,4148.21,4350.26,4586.15,4096.92,4641.03,4222.05,4238.46,4211.28,4280.51,4635.9,4393.85,0
4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.1,0
4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,0
4328.72,4011.79,4296.41,4155.9,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41,0
4326.15,4011.79,4292.31,4151.28,4347.69,4586.67,4095.9,4627.69,4210.77,4244.1,4212.82,4288.21,4632.82,4398.46,0
4321.03,4004.62,4284.1,4153.33,4345.64,4587.18,4093.33,4616.92,4202.56,4232.82,4209.74,4281.03,4628.21,4389.74,0
4319.49,4001.03,4280.51,4151.79,4343.59,4584.62,4089.74,4615.9,4212.31,4226.67,4201.03,4269.74,4625.13,4378.46,0
4325.64,4006.67,4278.46,4143.08,4344.1,4583.08,4087.18,4614.87,4205.64,4230.26,4195.9,4266.67,4622.05,4380.51,0
4326.15,4010.77,4276.41,4139.49,4345.13,4584.1,4091.28,4608.21,4187.69,4229.74,4202.05,4273.85,4627.18,4389.74,0
4326.15,4011.28,4276.92,4142.05,4344.1,4582.56,4092.82,4608.72,4194.36,4228.72,4212.82,4277.95,4637.44,4393.33,0


In [8]:
# the target is eyeDetection. Make sure it's binary valued
df['eyeDetection'].unique()


C1
0
1


In [9]:
# Convert the target to a logical factor
df['eyeDetection'] = df['eyeDetection'].asfactor()
df['eyeDetection'].nlevels()

[2]

In [10]:
# what are the levels
df['eyeDetection'].levels()

[['0', '1']]

In [12]:
# check NA in target
df['eyeDetection'].isna().sum()

0.0

In [14]:
# check if the target is unbalanced
df['eyeDetection'].table()

eyeDetection,Count
0,8257
1,6723


In [13]:
# check NA overall
df.isna().sum()

0.0

In [17]:
# Train, val, test split, by 0.6, 0.2, 0.2
train,test,valid = df.split_frame(ratios=[.6, .2])
train.shape

(9081, 15)

In [18]:
# See that the data is ready
train.describe()

Rows:9081
Cols:15




,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,eyeDetection
type,real,real,real,real,real,real,real,real,real,real,real,real,real,real,enum
mins,4198.97,2830.77,2457.44,4058.46,4304.62,4002.05,2086.15,4567.69,4147.69,1816.41,4103.59,2257.95,3504.1,4212.31,
mean,4335.06918952,4009.74354256,4264.23381566,4193.08347649,4342.0783493,4620.52793965,4134.73943729,4615.98640899,4230.56185993,4230.87141724,4202.41012884,4279.06273869,4621.85382227,4373.87048343,
maxs,309231.0,5500.51,6880.51,642564.0,6474.36,8092.31,567179.0,5087.69,265641.0,4362.56,6823.08,6904.62,152308.0,121026.0,
sigma,3200.26368151,36.4318773049,39.7610794607,6699.72777882,30.5951767476,41.4957183741,5909.20404342,19.2757123879,2743.72690539,32.4777843215,42.1442765071,40.2895105133,1550.36846659,1224.88910586,
zeros,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,4329.23,4009.23,4289.23,4148.21,4350.26,4586.15,4096.92,4641.03,4222.05,4238.46,4211.28,4280.51,4635.9,4393.85,0
1,4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,0
2,4328.72,4011.79,4296.41,4155.9,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41,0


In [21]:
# list of features
x = list(train.columns)
del x[-1] # take the target out
x

[u'AF3',
 u'F7',
 u'F3',
 u'FC5',
 u'T7',
 u'P7',
 u'O1',
 u'O2',
 u'P8',
 u'T8',
 u'FC6',
 u'F4',
 u'F8',
 u'AF4']

In [22]:
# Run GBM

from h2o.estimators.gbm import H2OGradientBoostingEstimator

model = H2OGradientBoostingEstimator(distribution='bernoulli',
                                    ntrees=100,
                                    max_depth=4,
                                    learn_rate=0.1)

model.train(x=x, y='eyeDetection', training_frame=train, validation_frame=valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [23]:
print(model)

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1496780333520_1


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.109086743056
RMSE: 0.33028282283
LogLoss: 0.362473852773
Mean Per-Class Error: 0.138143334539
AUC: 0.941373743054
Gini: 0.882747486108
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.44148642815: 


,0,1,Error,Rate
0,4331.0,711.0,0.141,(711.0/5042.0)
1,548.0,3491.0,0.1357,(548.0/4039.0)
Total,4879.0,4202.0,0.1386,(1259.0/9081.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4414864,0.8472273,213.0
max f2,0.3172723,0.8918592,262.0
max f0point5,0.6101953,0.8734386,147.0
max accuracy,0.4490829,0.8623500,210.0
max precision,0.9877259,1.0,0.0
max recall,0.0862365,1.0,368.0
max specificity,0.9877259,1.0,0.0
max absolute_mcc,0.4490829,0.7221425,210.0
max min_per_class_accuracy,0.4447758,0.8607695,212.0
max mean_per_class_accuracy,0.4490829,0.8618567,210.0


Gains/Lift Table: Avg response rate: 44.48 %



...

,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0102424,0.9670306,2.1648189,2.1648189,1.0,1.0,0.0221729,0.0221729,116.4818921,116.4818921
,2,0.0201434,0.9510694,2.1648189,2.1648189,1.0,1.0,0.0214339,0.0436068,116.4818921,116.4818921
,3,0.0300444,0.9409827,2.1648189,2.1648189,1.0,1.0,0.0214339,0.0650407,116.4818921,116.4818921
,4,0.0402868,0.9314097,2.1648189,2.1648189,1.0,1.0,0.0221729,0.0872136,116.4818921,116.4818921
,5,0.0501878,0.9232208,2.1648189,2.1648189,1.0,1.0,0.0214339,0.1086475,116.4818921,116.4818921
,6,0.1000341,0.8703005,2.0906813,2.1278766,0.9657534,0.9829352,0.1042129,0.2128603,109.0681287,112.7876619
,7,0.1502219,0.8177315,2.1648189,2.1402187,1.0,0.9886364,0.1086475,0.3215078,116.4818921,114.0218706
,8,0.2000683,0.7585810,2.0906813,2.1278766,0.9657534,0.9829352,0.1042129,0.4257206,109.0681287,112.7876619
,9,0.3001024,0.6419445,1.8249497,2.0269010,0.8430034,0.9362912,0.1825573,0.6082779,82.4949739,102.6900992
,10,0.4001366,0.5222092,1.5515767,1.9080699,0.7167235,0.8813993,0.1552106,0.7634885,55.1576701,90.8069919



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2017-06-06 16:41:49,0.016 sec,0.0,0.4969408,0.6870351,0.5,1.0,0.5552252,0.4988439,0.6908407,0.5,1.0,0.5380676
,2017-06-06 16:41:49,0.245 sec,1.0,0.4884152,0.6700350,0.7376888,1.9439699,0.3747385,0.4908356,0.6748100,0.7225467,1.7207535,0.3810174
,2017-06-06 16:41:50,0.344 sec,2.0,0.4824771,0.6582918,0.7618535,1.8990738,0.3289285,0.4851397,0.6635016,0.7473367,1.7875476,0.3270741
,2017-06-06 16:41:50,0.394 sec,3.0,0.4769029,0.6472687,0.7713225,2.0508835,0.3370774,0.4801126,0.6535277,0.7592167,1.9079760,0.3424377
,2017-06-06 16:41:50,0.456 sec,4.0,0.4705469,0.6348550,0.7889779,2.1562919,0.3333333,0.4741644,0.6418808,0.7754154,2.0875040,0.3414135
---,---,---,---,---,---,---,---,---,---,---,---,---,---
,2017-06-06 16:41:53,3.699 sec,52.0,0.3723546,0.4419510,0.9033239,2.2483288,0.1813677,0.3797218,0.4551600,0.8949779,2.1648189,0.2024582
,2017-06-06 16:41:53,3.782 sec,53.0,0.3716132,0.4405494,0.9041503,2.2483288,0.1812576,0.3790407,0.4539096,0.8959991,2.1648189,0.1952885
,2017-06-06 16:41:53,3.877 sec,54.0,0.3705788,0.4385553,0.9053187,2.2483288,0.1805969,0.3780809,0.4520594,0.8972499,2.1648189,0.1935814
,2017-06-06 16:41:53,3.975 sec,55.0,0.3692234,0.4359509,0.9066114,2.2483288,0.1789451,0.3769242,0.4498020,0.8984786,2.1648189,0.1942643



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
O1,1169.9899902,1.0,0.1790857
P7,1091.3742676,0.9328065,0.1670523
F7,824.2851562,0.7045233,0.1261701
F8,589.1867676,0.5035827,0.0901845
AF3,479.7887573,0.4100794,0.0734394
AF4,391.3667297,0.3345043,0.0599050
F4,354.5749512,0.3030581,0.0542734
O2,343.7891846,0.2938394,0.0526224
T8,302.3771362,0.2584442,0.0462837
T7,279.0652466,0.2385193,0.0427154


In [24]:
'''
model performance on test set
'''
perf = model.model_performance(test)
perf.show()


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.125320091361
RMSE: 0.354005778712
LogLoss: 0.402251894252
Mean Per-Class Error: 0.16903322425
AUC: 0.912928390394
Gini: 0.825856780788
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.46009850431: 


,0,1,Error,Rate
0,1389.0,250.0,0.1525,(250.0/1639.0)
1,249.0,1082.0,0.1871,(249.0/1331.0)
Total,1638.0,1332.0,0.168,(499.0/2970.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4600985,0.8126173,209.0
max f2,0.2604361,0.8674863,293.0
max f0point5,0.5979298,0.8510029,153.0
max accuracy,0.5160789,0.8363636,184.0
max precision,0.9863125,1.0,0.0
max recall,0.0826836,1.0,373.0
max specificity,0.9863125,1.0,0.0
max absolute_mcc,0.5160789,0.6698244,184.0
max min_per_class_accuracy,0.4373378,0.8273337,217.0
max mean_per_class_accuracy,0.4721350,0.8309668,204.0


Gains/Lift Table: Avg response rate: 44.81 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0101010,0.9672950,2.2314050,2.2314050,1.0,1.0,0.0225394,0.0225394,123.1404959,123.1404959
,2,0.0202020,0.9570851,2.2314050,2.2314050,1.0,1.0,0.0225394,0.0450789,123.1404959,123.1404959
,3,0.0303030,0.9464572,2.2314050,2.2314050,1.0,1.0,0.0225394,0.0676183,123.1404959,123.1404959
,4,0.0400673,0.9340921,2.2314050,2.2314050,1.0,1.0,0.0217881,0.0894065,123.1404959,123.1404959
,5,0.0501684,0.9245583,2.2314050,2.2314050,1.0,1.0,0.0225394,0.1119459,123.1404959,123.1404959
,6,0.1,0.8664410,2.1861738,2.2088655,0.9797297,0.9898990,0.1089406,0.2208866,118.6173777,120.8865515
,7,0.1501684,0.8089331,2.1565256,2.1913798,0.9664430,0.9820628,0.1081893,0.3290759,115.6525598,119.1379758
,8,0.2,0.7464477,2.1861738,2.1900826,0.9797297,0.9814815,0.1089406,0.4380165,118.6173777,119.0082645
,9,0.3,0.6319283,1.8632607,2.0811420,0.8350168,0.9326599,0.1863261,0.6243426,86.3260706,108.1141999
,10,0.4,0.5048208,1.3673929,1.9027047,0.6127946,0.8526936,0.1367393,0.7610819,36.7392938,90.2704733


In [26]:
'''
k-fold Cross validation model performance
We can simply pass the whole dataset and not assign a validation set
'''
cvmodel = H2OGradientBoostingEstimator(distribution='bernoulli',
                                       ntrees=100,
                                       max_depth=4,
                                       learn_rate=0.1,
                                       nfolds=5)

cvmodel.train(x=x, y='eyeDetection', training_frame=df)

gbm Model Build progress: |████████████████████████████████████████████████| 100%


In [27]:
cvmodel.auc

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1496780333520_231


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.115469077993
RMSE: 0.339807413094
LogLoss: 0.37998095395
Mean Per-Class Error: 0.144973382331
AUC: 0.934354672738
Gini: 0.868709345476
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.425021052762: 


,0,1,Error,Rate
0,6904.0,1353.0,0.1639,(1353.0/8257.0)
1,849.0,5874.0,0.1263,(849.0/6723.0)
Total,7753.0,7227.0,0.147,(2202.0/14980.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4250211,0.8421505,218.0
max f2,0.3045416,0.8887688,268.0
max f0point5,0.5744817,0.8659983,162.0
max accuracy,0.4581327,0.8563418,206.0
max precision,0.9866479,1.0,0.0
max recall,0.0565259,1.0,381.0
max specificity,0.9866479,1.0,0.0
max absolute_mcc,0.4581327,0.7097648,206.0
max min_per_class_accuracy,0.4443418,0.8534577,211.0
max mean_per_class_accuracy,0.4581327,0.8550266,206.0


Gains/Lift Table: Avg response rate: 44.88 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error
,2017-06-07 09:59:23,7.081 sec,0.0,0.4973715,0.6878948,0.5,1.0,0.5512016
,2017-06-07 09:59:23,7.094 sec,1.0,0.4905016,0.6741595,0.7021724,1.9821161,0.3208278
,2017-06-07 09:59:23,7.105 sec,2.0,0.4849382,0.6631539,0.7561982,2.1369602,0.3379172
,2017-06-07 09:59:23,7.117 sec,3.0,0.4783284,0.6502182,0.7758654,2.1836085,0.3164887
,2017-06-07 09:59:23,7.131 sec,4.0,0.4742147,0.6421131,0.7901831,2.1839036,0.3248999
---,---,---,---,---,---,---,---,---
,2017-06-07 09:59:27,10.822 sec,72.0,0.3600516,0.4181045,0.9174780,2.2281719,0.1690921
,2017-06-07 09:59:27,10.898 sec,73.0,0.3589592,0.4159852,0.9183639,2.2281719,0.1706275
,2017-06-07 09:59:27,10.973 sec,74.0,0.3583546,0.4148197,0.9187967,2.2281719,0.1645527
,2017-06-07 09:59:27,11.050 sec,75.0,0.3577768,0.4136903,0.9193176,2.2281719,0.1640854



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
P7,2228.9809570,1.0,0.2161341
O1,1912.8731690,0.8581828,0.1854826
F7,1074.7462158,0.4821693,0.1042132
AF3,1005.1795654,0.4509592,0.0974677
F8,937.9238281,0.4207859,0.0909462
T7,501.3728333,0.2249337,0.0486158
FC6,460.0370789,0.2063890,0.0446077
T8,450.4796143,0.2021011,0.0436810
F4,377.0189514,0.1691441,0.0365578
F3,360.0674438,0.1615390,0.0349141


<bound method H2OGradientBoostingEstimator.auc of >

In [28]:
'''
Do a Grid search to optimize the parameters in the model
ntrees: Number of trees
max_depth: Maximum depth of a tree
learn_rate: Learning rate in the GBM
'''

ntrees_opt = [5,50,100]
max_depth_opt = [2,3,5]
learn_rate_opt = [0.1,0.2]

hyper_params = {'ntrees': ntrees_opt, 
                'max_depth': max_depth_opt,
                'learn_rate': learn_rate_opt}

In [29]:
'''
Define an "H2OGridSearch" object by specifying the algorithm (GBM) and the hyper parameters:
'''

from h2o.grid.grid_search import H2OGridSearch

gs = H2OGridSearch(H2OGradientBoostingEstimator, hyper_params = hyper_params)

In [31]:
'''
Train the model on the grid
'''
gs.train(x=x, y='eyeDetection', training_frame=train, validation_frame=valid)

gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [32]:
'''
Compare the models with different parameter values in the grid, ranked by AUC
'''
auc_table = gs.sort_by('auc(valid=True)',increasing=False)
print(auc_table)

[WARNING] in <ipython-input-32-079ea57dde9d> line 4:
    >>> auc_table = gs.sort_by('auc(valid=True)',increasing=False)
        ^^^^ grid.sort_by() is deprecated; use grid.get_grid() instead
Grid Search Results for H2OGradientBoostingEstimator: 


Model Id,"Hyperparameters: [learn_rate, ntrees, max_depth]",auc(valid=True)
Grid_GBM_py_17_sid_ae14_model_python_1496780333520_860_model_17,"[0.2, 100, 5]",0.9692184
Grid_GBM_py_17_sid_ae14_model_python_1496780333520_860_model_16,"[0.1, 100, 5]",0.9538964
Grid_GBM_py_17_sid_ae14_model_python_1496780333520_860_model_11,"[0.2, 50, 5]",0.9488988
Grid_GBM_py_17_sid_ae14_model_python_1496780333520_860_model_10,"[0.1, 50, 5]",0.9259589
Grid_GBM_py_17_sid_ae14_model_python_1496780333520_860_model_15,"[0.2, 100, 3]",0.9126445
Grid_GBM_py_17_sid_ae14_model_python_1496780333520_860_model_14,"[0.1, 100, 3]",0.8816357
Grid_GBM_py_17_sid_ae14_model_python_1496780333520_860_model_9,"[0.2, 50, 3]",0.8795933
Grid_GBM_py_17_sid_ae14_model_python_1496780333520_860_model_5,"[0.2, 5, 5]",0.8474761
Grid_GBM_py_17_sid_ae14_model_python_1496780333520_860_model_8,"[0.1, 50, 3]",0.8436301
Grid_GBM_py_17_sid_ae14_model_python_1496780333520_860_model_4,"[0.1, 5, 5]",0.8325584


In [33]:
'''
Check the best model's performance from the AUC table on the test set
'''
best_model = h2o.get_model(auc_table['Model Id'][0])
best_perf = best_model.model_performance(test)
best_perf.auc()

0.9635355618519107